## Preparation of final dataset
1. Loading files for Transport and Industry
2. Process industry data per subsector (Steel, Basic Chemicals, NMM)
3. Process transport data per subsectors and fuels (Electricity, Hydrogen, Gases, Liquids)
4. Prepare final excel file with one per each year and country merging all values

In [1035]:
import pandas as pd
from collections import defaultdict
import os

1. Loading files for Transport and Industry

In [1036]:
transport_file = r"C:\Users\mar.eco\OneDrive - CBS - Copenhagen Business School\Desktop\PtX-Markets\REMIND\Results_REMIND_JRC.csv"
industry_path = r"C:\Users\mar.eco\OneDrive - CBS - Copenhagen Business School\Desktop\PtX-Markets\Scripts\Industry\Results_per_Country"

In [1037]:
country_code = "LT"
# ['EU27', 'AT', 'BE', 'BG', 'CY', 
# 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 
# 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 
# 'PT', 'RO', 'SE', 'SI', 'SK']

years = [2030, 2040, 2050]

In [1038]:
output_path = rf"C:\Users\mar.eco\OneDrive - CBS - Copenhagen Business School\Desktop\PtX-Markets\Outputs\PtX_demand_{country_code}.xlsx"

In [1039]:
os.chdir(r"C:\Users\mar.eco\OneDrive - CBS - Copenhagen Business School\Desktop\PtX-Markets")

In [1040]:
def load_transport_data(filepath):
    df = pd.read_csv(filepath)
    df['Year'] = df['Year'].astype(int)
    return df

In [1041]:
def load_industry_data(filepath):
    industry_data = []
    industry_files = [f for f in os.listdir(filepath) if f.endswith(".xlsx")]

    for file_name in industry_files:
        year, country = file_name.replace(".xlsx", "").split("_")
        file_path = os.path.join(filepath, file_name)
        df = pd.read_excel(file_path, index_col=0)
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)

        for material in df.index:
            for sector in df.columns:
                industry_data.append({
                    "Year": int(year),
                    "Country": country,
                    "Category": sector,
                    "Material": material.strip(),
                    "Value": df.loc[material, sector] * 3.6 * 0.000001
                })

    return pd.DataFrame(industry_data)

In [1042]:
transport_df = load_transport_data(transport_file)
industry_df = load_industry_data(industry_path)

2. Process industry data per subsector (Steel, Basic Chemicals, NMM)

In [1043]:
industry_mapping = {
    'Overall Demand': 'Overall Demand',
    'Hydrogen': 'Hydrogen',
    'Ammonia': 'Ammonia',
    'Methanol': 'Methanol',
    'Biomass': 'Biomass [Solid]',
    'Biogas': 'Biogenic Gases',
    'Other': 'Renewable Energy Carrier',
}

In [1044]:
industry_output = defaultdict(lambda: defaultdict(float))

In [1045]:
industry_filtered = industry_df[industry_df["Country"] == country_code]

In [1046]:

for _, row in industry_filtered.iterrows():
    year = row["Year"]
    mat = row["Material"]
    fuel = industry_mapping.get(mat)
    if fuel:
        key = (fuel, year)
        industry_output[key][row["Category"]] += row["Value"]

In [1047]:
industry_rows = []

for (fuel_group, year), sectors in industry_output.items():
    for sector, value in sectors.items():
        industry_rows.append({
            "FuelGroup": fuel_group,
            "Year": year,
            "IndustrySector": sector,
            "Value": value if value != 0 else None 
        })

industry_df = pd.DataFrame(industry_rows)

In [1048]:
pivot_industry = industry_df.pivot_table(
    index=["FuelGroup", "Year"],
    columns="IndustrySector",
    values="Value",
    aggfunc="sum"
).reset_index()
pivot_industry

IndustrySector,FuelGroup,Year,Chemicals,Iron & steel,Non-metallic minerals
0,Ammonia,2030,0.014032,0.0,0.000000
1,Ammonia,2040,0.012921,0.0,0.000000
2,Ammonia,2050,0.013004,0.0,0.000000
3,Biogenic Gases,2030,0.000000,0.0,0.000098
4,Biogenic Gases,2040,0.000000,0.0,0.000417
5,Biogenic Gases,2050,0.000000,0.0,0.001038
6,Biomass [Solid],2030,0.000000,0.0,0.000471
7,Biomass [Solid],2040,0.000000,0.0,0.000456
8,Biomass [Solid],2050,0.000000,0.0,0.000458
9,Hydrogen,2030,0.000000,0.0,0.000000


3. Process transport data per subsectors and fuels (Electricity, Hydrogen, Gases, Liquids)

In [1049]:
fuel_map = {
    'Liquids|Biomass': 'Biogenic Liquids',
    'Gases|Biomass': 'Biogenic Gases',
    'Liquids|Hydrogen': 'Synthetic Liquids',
    'Gases|Hydrogen': 'Synthetic Gases',
    'Hydrogen': 'Hydrogen',
}


In [1050]:
transport_output = defaultdict(lambda: defaultdict(float))

In [1051]:
categories = [
    "FE|Transport|Freight|Road|Heavy",
    "FE|Transport|Freight|Road|Light",
    "FE|Transport|Pass|Road|Bus",
    "FE|Transport|Pass|Road|LDV|Four Wheelers",
    "FE|Transport|Pass|Road|LDV|Two Wheelers",
    "FE|Transport|Pass|Domestic Aviation",
    "FE|Transport|Pass|Aviation",
    "FE|Transport|Pass|Rail",
    "FE|Transport|Freight|Rail",
    "FE|Transport|Bunkers|Freight|International Shipping",
    "FE|Transport|Freight|Domestic Shipping"               
]

transport_fuel_paths = [
    # Freight Road Heavy
    "FE|Transport|Freight|Road|Heavy|Electricity",
    "FE|Transport|Freight|Road|Heavy|Hydrogen",
    "FE|Transport|Freight|Road|Heavy|Gases",
    "FE|Transport|Freight|Road|Heavy|Gases|Biomass",
    "FE|Transport|Freight|Road|Heavy|Gases|Fossil",
    "FE|Transport|Freight|Road|Heavy|Gases|Hydrogen",
    "FE|Transport|Freight|Road|Heavy|Liquids",
    "FE|Transport|Freight|Road|Heavy|Liquids|Biomass",
    "FE|Transport|Freight|Road|Heavy|Liquids|Fossil",
    "FE|Transport|Freight|Road|Heavy|Liquids|Hydrogen",

    # Freight Road Light
    "FE|Transport|Freight|Road|Light|Electricity",
    "FE|Transport|Freight|Road|Light|Hydrogen",
    "FE|Transport|Freight|Road|Light|Gases",
    "FE|Transport|Freight|Road|Light|Gases|Biomass",
    "FE|Transport|Freight|Road|Light|Gases|Fossil",
    "FE|Transport|Freight|Road|Light|Gases|Hydrogen",
    "FE|Transport|Freight|Road|Light|Liquids",
    "FE|Transport|Freight|Road|Light|Liquids|Biomass",
    "FE|Transport|Freight|Road|Light|Liquids|Fossil",
    "FE|Transport|Freight|Road|Light|Liquids|Hydrogen",

    # Passenger Road Bus
    "FE|Transport|Pass|Road|Bus|Electricity",
    "FE|Transport|Pass|Road|Bus|Hydrogen",
    "FE|Transport|Pass|Road|Bus|Gases",
    "FE|Transport|Pass|Road|Bus|Gases|Biomass",
    "FE|Transport|Pass|Road|Bus|Gases|Fossil",
    "FE|Transport|Pass|Road|Bus|Gases|Hydrogen",
    "FE|Transport|Pass|Road|Bus|Liquids",
    "FE|Transport|Pass|Road|Bus|Liquids|Biomass",
    "FE|Transport|Pass|Road|Bus|Liquids|Fossil",
    "FE|Transport|Pass|Road|Bus|Liquids|Hydrogen",

    # Passenger Road LDV Four Wheelers
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Electricity",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Gases",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Gases|Biomass",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Gases|Fossil",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Gases|Hydrogen",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Liquids",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Liquids|Biomass",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Liquids|Fossil",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Liquids|Hydrogen",

    # Passenger Road LDV Two Wheelers
    "FE|Transport|Pass|Road|LDV|Two Wheelers|Electricity",
    "FE|Transport|Pass|Road|LDV|Two Wheelers|Liquids",
    "FE|Transport|Pass|Road|LDV|Two Wheelers|Liquids|Biomass",
    "FE|Transport|Pass|Road|LDV|Two Wheelers|Liquids|Fossil",
    "FE|Transport|Pass|Road|LDV|Two Wheelers|Liquids|Hydrogen",

    # Bunkers Freight International Shipping
    "FE|Transport|Bunkers|Freight|International Shipping|Liquids",

    # Freight Domestic Shipping
    "FE|Transport|Freight|Domestic Shipping|Liquids",
    "FE|Transport|Freight|Domestic Shipping|Liquids|Biomass",
    "FE|Transport|Freight|Domestic Shipping|Liquids|Fossil",
    "FE|Transport|Freight|Domestic Shipping|Liquids|Hydrogen",

    # Bunkers Pass International Aviation
    "FE|Transport|Bunkers|Pass|International Aviation|Liquids",

    # Passenger Domestic Aviation
    "FE|Transport|Pass|Domestic Aviation|Hydrogen",
    "FE|Transport|Pass|Domestic Aviation|Liquids",
    "FE|Transport|Pass|Domestic Aviation|Liquids|Biomass",
    "FE|Transport|Pass|Domestic Aviation|Liquids|Fossil",
    "FE|Transport|Pass|Domestic Aviation|Liquids|Hydrogen",

    # Passenger Aviation
    "FE|Transport|Pass|Aviation|Hydrogen",
    "FE|Transport|Pass|Aviation|Liquids",
    "FE|Transport|Pass|Aviation|Liquids|Biomass",
    "FE|Transport|Pass|Aviation|Liquids|Fossil",
    "FE|Transport|Pass|Aviation|Liquids|Hydrogen",

    # Passenger Rail
    "FE|Transport|Pass|Rail|Hydrogen",
    "FE|Transport|Pass|Rail|Liquids",
    "FE|Transport|Pass|Rail|Liquids|Biomass",
    "FE|Transport|Pass|Rail|Liquids|Fossil",
    "FE|Transport|Pass|Rail|Liquids|Hydrogen",

    # Freight Rail
    "FE|Transport|Freight|Rail|Hydrogen",
    "FE|Transport|Freight|Rail|Liquids", 
    "FE|Transport|Freight|Rail|Liquids|Biomass",
    "FE|Transport|Freight|Rail|Liquids|Fossil",
    "FE|Transport|Freight|Rail|Liquids|Hydrogen",
]

In [1052]:
def extract_main_and_fuel(category_str, categories):
    categories_sorted = sorted(categories, key=len, reverse=True)
    
    for cat_prefix in categories_sorted:
        if category_str.startswith(cat_prefix):
            # Fuel is whatever comes after the prefix (skip the '|')
            remainder = category_str[len(cat_prefix):]
            if remainder.startswith("|"):
                remainder = remainder[1:]  # remove leading '|'
            return cat_prefix, remainder
    # If no prefix matched, return None and full string as fuel
    return None, category_str

In [1053]:
transport_filtered = transport_df[(transport_df["Country"] == country_code) & 
                                  (transport_df["Year"].isin(years))]

In [1054]:
transport_filtered[["MainCategory", "Fuel"]] = transport_filtered["Category"].apply(lambda x: pd.Series(extract_main_and_fuel(x, categories)))
fuel_transport = transport_filtered[(transport_filtered["Fuel"].notna()) & 
                              (transport_filtered["Country"] == country_code)]

C:\Users\mar.eco\AppData\Local\Temp\ipykernel_15448\1015843666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transport_filtered[["MainCategory", "Fuel"]] = transport_filtered["Category"].apply(lambda x: pd.Series(extract_main_and_fuel(x, categories)))
C:\Users\mar.eco\AppData\Local\Temp\ipykernel_15448\1015843666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transport_filtered[["MainCategory", "Fuel"]] = transport_filtered["Category"].apply(lambda x: pd.Series(extract_main_and_fuel(x, categories

In [1055]:
main_category_mapping = {
    "FE|Transport|Freight|Road|Heavy": "Freight Road",
    "FE|Transport|Freight|Road|Light": "Freight Road",
    "FE|Transport|Pass|Road|Bus": "Pass Road",
    "FE|Transport|Pass|Road|LDV|Four Wheelers": "Pass Road",
    "FE|Transport|Pass|Road|LDV|Two Wheelers": "Pass Road",
    "FE|Transport|Pass|Domestic Aviation": "Pass Aviation",
    "FE|Transport|Pass|Aviation": "Pass Aviation",
    "FE|Transport|Pass|Rail": "Pass Rail",
    "FE|Transport|Freight|Rail": "Freight Rail",
    "FE|Transport|Bunkers|Freight|International Shipping": "Maritime",
    "FE|Transport|Freight|Domestic Shipping": "Maritime",
}

In [1056]:
fuel_transport["TransportSector"] = fuel_transport["MainCategory"].map(main_category_mapping)

In [1057]:
fuel_transport["FuelGroup"] = fuel_transport["Fuel"].map(fuel_map)

In [1058]:
fuel_transport = fuel_transport[fuel_transport["FuelGroup"].notna()]

In [1059]:
agg_transport = fuel_transport.groupby(
    ["Year", "FuelGroup", "TransportSector"]
)["Value"].sum().reset_index()

In [1060]:
pivot_transport = agg_transport.pivot_table(
    index=["FuelGroup", "Year"],
    columns="TransportSector",
    values="Value",
    aggfunc="sum"
).reset_index()
pivot_transport

TransportSector,FuelGroup,Year,Freight Rail,Freight Road,Maritime,Pass Aviation,Pass Rail,Pass Road
0,Biogenic Gases,2030,NaN,1.010880e-05,NaN,NaN,NaN,4.761503e-05
1,Biogenic Gases,2040,NaN,1.486320e-05,NaN,NaN,NaN,6.248113e-05
2,Biogenic Gases,2050,NaN,4.085164e-06,NaN,NaN,NaN,8.924191e-06
3,Biogenic Liquids,2030,3.263596e-04,1.591597e-03,3.283197e-03,0.000000e+00,4.448074e-06,3.487022e-03
4,Biogenic Liquids,2040,3.964739e-04,1.082973e-03,3.716634e-03,0.000000e+00,7.251095e-06,1.389709e-03
5,Biogenic Liquids,2050,5.099283e-04,2.817796e-04,5.293938e-03,0.000000e+00,1.295643e-05,8.375250e-05
6,Hydrogen,2030,NaN,4.024666e-05,NaN,3.133352e-10,NaN,2.039087e-04
7,Hydrogen,2040,NaN,6.312642e-05,NaN,2.622961e-08,NaN,9.889928e-04
8,Hydrogen,2050,NaN,1.016994e-04,NaN,4.445708e-08,NaN,1.367417e-03
9,Synthetic Gases,2030,NaN,0.000000e+00,NaN,NaN,NaN,0.000000e+00


4. Prepare final excel file with one per each year and country merging all values

In [1061]:
pivot_transport = pivot_transport.rename(columns={"TransportSector": "SectorGroup"})
pivot_industry = pivot_industry.rename(columns={"FuelGroup": "FuelGroup", "Year": "Year"})

In [1062]:
pivot_industry["SectorGroup"] = "Industry"
pivot_transport["SectorGroup"] = "Transport"

In [1063]:
column_order = [
    "Iron & steel", "Chemicals", "Non-metallic minerals",  # Industry
    "Pass Road", "Pass Rail", "Pass Aviation",             # Transport (passenger)
    "Freight Road", "Freight Rail",                        # Transport (freight)
    "Maritime"                                             # Maritime (bunkers)
]

In [1064]:
all_columns = set(column_order + ["FuelGroup", "Year", "SectorGroup"])

# Add missing columns as None
for col in all_columns:
    if col not in pivot_transport.columns:
        pivot_transport[col] = None
    if col not in pivot_industry.columns:
        pivot_industry[col] = None

In [1065]:
combined_df = pd.concat([pivot_transport, pivot_industry], ignore_index=True)

C:\Users\mar.eco\AppData\Local\Temp\ipykernel_15448\4054876749.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([pivot_transport, pivot_industry], ignore_index=True)


In [1066]:
fuel_order = [
    "Overall Demand", "Hydrogen", "Methanol", "Ammonia", "Synthetic Gases",
    "Biogenic Gases", "Synthetic Liquids", "Biogenic Liquids",
    "Biomass [Solid]", "Renewable Energy Carrier"
]

In [1067]:
fuel_order_map = {fuel: i for i, fuel in enumerate(fuel_order)}
combined_df["FuelSortOrder"] = combined_df["FuelGroup"].map(fuel_order_map)

In [1068]:
final_df = combined_df.drop(columns=["SectorGroup", "FuelSortOrder"])  # drop SectorGroup
final_df = final_df.groupby(["FuelGroup", "Year"], as_index=False).sum()

In [1069]:
def clean_zeros_and_nans(df):
    return df.applymap(lambda x: "" if pd.isna(x) or x == 0 else x)

In [1070]:
clean_df = clean_zeros_and_nans(final_df)

C:\Users\mar.eco\AppData\Local\Temp\ipykernel_15448\2748423523.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: "" if pd.isna(x) or x == 0 else x)


In [1071]:
ordered_cols = ["FuelGroup", "Year"] + column_order
final_correct_order = clean_df[ordered_cols]
final_correct_order["FuelSortOrder"] = final_correct_order["FuelGroup"].map(fuel_order_map)
final_correct_order = final_correct_order.sort_values(by=["Year", "FuelSortOrder"]).drop(columns=["FuelSortOrder"])
final_correct_order

C:\Users\mar.eco\AppData\Local\Temp\ipykernel_15448\2721408278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_correct_order["FuelSortOrder"] = final_correct_order["FuelGroup"].map(fuel_order_map)


,FuelGroup,Year,Iron & steel,Chemicals,Non-metallic minerals,Pass Road,Pass Rail,Pass Aviation,Freight Road,Freight Rail,Maritime
18,Overall Demand,2030,,,,,,,,,
12,Hydrogen,2030,,,,0.000204,,0.0,0.00004,,
15,Methanol,2030,,,,,,,,,
0,Ammonia,2030,,0.014032,,,,,,,
24,Synthetic Gases,2030,,,,,,,,,
3,Biogenic Gases,2030,,,0.000098,0.000048,,,0.00001,,
27,Synthetic Liquids,2030,,,,,,,,,
6,Biogenic Liquids,2030,,,,0.003487,0.000004,,0.001592,0.000326,0.003283
9,Biomass [Solid],2030,,,0.000471,,,,,,
21,Renewable Energy Carrier,2030,,,0.000394,,,,,,


In [1072]:
final_correct_order.to_excel(output_path, index=False, na_rep="")